Lending Club releases data for all of the approved and declined loan applications periodically on their [website](https://www.lendingclub.com/info/download-data.action). Different year ranges can be selected to download the datasets (in CSV format) for both approved and declined loans.

The LoanStats sheet describes the approved loans datasets and the RejectStats describes the rejected loans datasets. Since rejected applications don't appear on the Lending Club marketplace and aren't available for investment, we'll be focusing on data on approved loans only.

**GOAL - Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?**

We will focus on data for approved loans from 2007 to 2011, since a good number of the loans have already been paid off. In the datasets for later years, many of the loans are current and still being paid off.

We reduced the size of LoanStats3a.csv by:
removing the first line, because it contains the extraneous text Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action) instead of the column titles, which prevents the dataset from being parsed by the pandas library properly  
removing the desc column, which contains a long text explanation for each loan  
removing the url column, which contains a link to each loan on Lending Club which can only be accessed with an investor account  
removing all columns containing more than 50% missing values, which allows us to move faster since we can spend less time trying to fill these values

and named the filtered dataset loans_2007.csv

In [1]:
import pandas as pd
import numpy as np

# the instructions to clean original dataset did not work so downloaded the csv from the mission and deleted last 2 lines which had some descr stats
loans_2007 = pd.read_csv('loans_2007.csv')
print(loans_2007.shape)
loans_2007.head()

(42536, 52)


C:\Users\NS-PC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-13,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [2]:
loans_2007.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42536 entries, 0 to 42535
Data columns (total 52 columns):
id                            42536 non-null object
member_id                     42535 non-null float64
loan_amnt                     42535 non-null float64
funded_amnt                   42535 non-null float64
funded_amnt_inv               42535 non-null float64
term                          42535 non-null object
int_rate                      42535 non-null object
installment                   42535 non-null float64
grade                         42535 non-null object
sub_grade                     42535 non-null object
emp_title                     39909 non-null object
emp_length                    41423 non-null object
home_ownership                42535 non-null object
annual_inc                    42531 non-null float64
verification_status           42535 non-null object
issue_d                       42535 non-null object
loan_status                   42535 non-null object
p

The Dataframe contains many columns and it can be cumbersome to explore it all at once. Let's break up the columns into 3 groups of 18 columns and use the data dictionary to become familiar with what each column represents. 

We want to pay attention to any features that:
* leak information from the future (after the loan has already been funded)
* don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
* formatted poorly and need to be cleaned up
* require more data or a lot of processing to turn into a useful feature
* contain redundant information

We need to especially pay attention to data leakage, since it can cause our model to overfit. This is because the model would be using data about the target column that wouldn't be available when we're using the model on future loans.

# Dropping columns

In [3]:
# 1st set of cols to drop
loans_2007 = loans_2007.drop(['id','member_id','funded_amnt','funded_amnt_inv','grade','sub_grade','emp_title','issue_d'],axis=1)
print(loans_2007.shape)

(42536, 44)


In [4]:
# 2nd set of cols to drop
loans_2007 = loans_2007.drop(['zip_code','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp'],axis=1)
print(loans_2007.shape)

(42536, 38)


In [5]:
# 3rd set of cols to drop
loans_2007 = loans_2007.drop(['total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt'],axis=1)
print(loans_2007.shape)

(42536, 32)


# Target column

We should use the **loan_status column**, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and we need to convert it to a numerical one for training a model.  

Let's explore the different values in this column and come up with a strategy for converting the values in this column.

In [6]:
loans_2007['loan_status'].value_counts(dropna=False).sort_values()

NaN                                                        1
Default                                                    3
Late (16-30 days)                                          8
In Grace Period                                           20
Late (31-120 days)                                        24
Does not meet the credit policy. Status:Charged Off      761
Current                                                  961
Does not meet the credit policy. Status:Fully Paid      1988
Charged Off                                             5634
Fully Paid                                             33136
Name: loan_status, dtype: int64

| Loan Status                                         | Count | Meaning                                                                                                                                           |
|-----------------------------------------------------|-------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| Fully Paid                                          | 33136 | Loan has been fully paid off.                                                                                                                     |
| Charged Off                                         | 5634  | Loan for which there is no longer a reasonable expectation of further payments.                                                                   |
| Does not meet the credit policy. Status:Fully Paid  | 1988  | While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.    |
| Does not meet the credit policy. Status:Charged Off | 761   | While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace. |
| In Grace Period                                     | 20    | The loan is past due but still in the grace period of 15 days.                                                                                    |
| Late (16-30 days)                                   | 8     | Loan hasn't been paid in 16 to 30 days (late on the current payment).                                                                             |
| Late (31-120 days)                                  | 24    | Loan hasn't been paid in 31 to 120 days (late on the current payment).                                                                            |
| Current                                             | 961   | Loan is up to date on current payments.                                                                                                           |
| Default                                             | 3     | Loan is defaulted on and no payment has been made for more than 121 days.                                                                         |

From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't. **Only the Fully Paid and Charged Off values describe the final outcome of the loan.** The other values describe loans that are still being pain and whetherthe borrower will pay back the loan on time or not. While the Default status resembles the Charged Off status, for Lending Club's, loans that are charged off have essentially no chance of being repaid while default ones have a small chance. 

Since we're interested in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a binary classification one. 
Let's remove all the loans that don't contain either Fully Paid and Charged Off as the loan's status and then transform the Fully Paid values to 1 for the positive case and the Charged Off values to 0 for the negative case. 

We also need to keep in mind **class imbalance** between the positive and negative cases. While there are 33,136 loans that have been fully paid off, there are only 5,634 that were charged off. This class imbalance is a common problem in binary classification and during training, the model ends up having a strong bias towards predicting the class with more observations in the training set and will rarely predict the class with less observations. The stronger the imbalance, the more biased the model becomes. We will explore this later.

# Binary Classification

* Remove all rows from loans_2007 that contain values other than Fully Paid or Charged Off for the loan_status column.
* Replace:
  * Fully Paid with 1
  * Charged Off with 0

In [7]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == 'Fully Paid') | (loans_2007['loan_status'] == 'Charged Off')]
status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

loans_2007 = loans_2007.replace(status_replace) 

# Removing single value columns

In [8]:
orig_columns = loans_2007.columns
drop_columns = []
for col in orig_columns:
    col_series = loans_2007[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans_2007 = loans_2007.drop(drop_columns, axis=1)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']
